# irradiance_comparison.ipynb

**Recreate the graphs from Thesis §3.4 “Directional response characterisation against pyrheliometer and shaded pyranometer: results”**

This notebook reads the precomputed CSV files from the folders:

- `Directional_response_outdoor_tilted/`
- `Directional_response_outdoor/`

and regenerates all the figures listed by Francesco (filenames are used as figure titles and output names).

IMPORTANT NOTICE: this is a quick and dirty job, there may be duplicates in the code.

In [14]:
# --- Configuration -----------------------------------------------------------
# --- Configuration -----------------------------------------------------------
import sys
PATH_TO_MODULES = r"C:/Users/wsfm/OneDrive - Loughborough University/_Personal_Backup/ground_based_solar_irradiance/"
#adding path to the tailored made modules
sys.path.append(PATH_TO_MODULES)

In [15]:
import v_results_3_source_py.irr_plot as irplt

In [16]:
import v_results_3_source_py.irr_io as irio

In [24]:
import v_results_3_source_py.plot_helper as plthlp

In [18]:
from pathlib import Path
import pandas as pd
import numpy as np
import os

from typing import Dict, List, Tuple, Optional
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from matplotlib.font_manager import FontProperties

#loc giving freedom since change depending how many graphs
# Loc can also be a 2-tuple giving the coordinates of the lower-left corner of the legend in axes coordinates
# (in which case bbox_to_anchor will be ignored). @FM does not happen
# https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.legend.html
_DEFAULT_KWARGS ={plt.legend: {'loc': (0, 0), #'best' 0, 'upper right' 1, 'upper left' 2, 'lower left' 3, 'lower right' 4,
                 #'right' 5, 'center left'	6, 'center right' 7, 'lower center'	8, 'upper center' 9, 'center' 10
                 #loc position change starting from bbox_to_anchor. l/r & u/l seems inverted when bbox_to_anchor
                 'bbox_to_anchor': (0, -1)  # box location (x,y) or (x, y, width, height)
                 }}


# Hot-patch the loaded irr_plot module so Path is defined in its globals
irplt.Path = Path  # inject Path into irr_plot's module namespace

LGN_BBOX_TO_ANCHOR =  _DEFAULT_KWARGS[plt.legend]['bbox_to_anchor']
LGN_LOC =  _DEFAULT_KWARGS[plt.legend]['loc']


# Base directory containing the two folders
BASE_DIR = r"C:\Users\wsfm\OneDrive - Loughborough University\_Personal_Backup\ground_based_solar_irradiance" # <-- set this to the project root if needed

# Input folders
DIR_TILTED = os.path.join(BASE_DIR,'assets','Directional_response_outdoor_tilted')
DIR_OUTDOOR = os.path.join(BASE_DIR, 'assets','Directional_response_outdoor')

# Output folder for figures
OUT_DIR = os.path.join(BASE_DIR,"v_characterisation_output")
OUT_DIR_TILTED = os.path.join(BASE_DIR,"v_characterisation_output_tilted")

# File name lists (stems without .csv)
FILES_TILTED = [
    'crest_ground_180824_29_error_abs_bi200cr20',
    'crest_ground_180824_29_error_cos_bi200cr20',
    'crest_ground_180824_29_error_tcorrected_abs_bi200cr20',
    'crest_ground_180824_29_error_tcorrected_cos_bi200cr20',
]

FILES_OUTDOOR = [
    'crest_roof_180825_abs_bi200cr20',
    'crest_roof_180825_cos_bi200cr20',
    'crest_roof_almostclear_abs_bi200cr20',
    'crest_roof_almostclear_cos_bi200cr20',
    'crest_roof_months_all_abs_bi200cr20',
    'crest_roof_months_all_cos_bi200cr20',
    'eurac_2015_error_abs_bi200cr20',
    'eurac_2015_error_cos_bi200cr20'
]

SUFFIXES = [""," nonvent", " ventilated", ]

TESTS = ["onefig"]
TESTS = []

In [19]:
# --- Configuration -----------------------------------------------------------
import sys
PATH_TO_MODULES = r"C:/Users/wsfm/OneDrive - Loughborough University/_Personal_Backup/ground_based_solar_irradiance/"
#adding path to the tailored made modules
sys.path.append(PATH_TO_MODULES)

In [20]:
#excluding store to verify if cause problems
#retrieving stored variables
%store -r
#automatically reload
%load_ext autoreload
# autoreload 0 if deactivated 2 if all modules but excluded -
%autoreload 1
#interrupt when needed
%aimport v_characterisation_source_py.irr_plot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# --- Discover CSV files ------------------------------------------------------
found_tilted = irio.find_csvs(DIR_TILTED, FILES_TILTED)
found_out = irio.find_csvs(DIR_OUTDOOR, FILES_OUTDOOR)

missing_tilted = [n for n in FILES_TILTED if n not in found_tilted]
missing_out = [n for n in FILES_OUTDOOR if n not in found_out]

print('Found tilted: ', len(found_tilted), 'Missing:', missing_tilted)
print('Found outdoor:', len(found_out), 'Missing:', missing_out)

Found tilted:  4 Missing: []
Found outdoor: 8 Missing: []


In [11]:
# --- Process tilted files ----------------------------------------------------
for name, path in found_tilted.items():
    # TEST 19/10/25
    if name in ["crest_ground_180824_29_error_tcorrected_cos_bi200cr20"]:
        df = irio.load_csv(path)
        cos_clms  = [c for c in df.columns.to_list() if "cos" in c]
        for c in cos_clms:
            df_dsc = df[c].describe(percentiles=[0.05,0.95])
            df = df.loc[(df[c]<df_dsc["95%"])&(df[c]>df_dsc["5%"]),:] 
        plthlp.make_figures_for_df(df, name, OUT_DIR_TILTED)
        if "onefig" in TESTS: break
print('Tilted figures saved to', OUT_DIR_TILTED)

Tilted figures saved to C:\Users\wsfm\OneDrive - Loughborough University\_Personal_Backup\ground_based_solar_irradiance\v_characterisation_output_tilted


In [47]:
TESTS = ["onefig"]

In [28]:
# --- Process outdoor files ---------------------------------------------------
for name, path in found_out.items():
    if name == 'crest_roof_180825_cos_bi200cr20':
    # 19/10/25 excluded too many data and cannot calculate directional response
    # if name == "crest_roof_months_all_abs_bi200cr20": 
    # if name == "crest_roof_almostclear_abs_bi200cr20": 
        df = irio.load_csv(path)
        if name == "crest_roof_months_all_abs_bi200cr20":
            df = df[(df["abs_err"]<=30)&(df["abs_err"]>=-30)]


        cos_clms  = [c for c in df.columns.to_list() if "cos" in c]
        if name != "crest_roof_almostclear_abs_bi200cr20": 
            for c in cos_clms:
                df_dsc = df[c].describe(percentiles=[0.05,0.95])
                df = df.loc[(df[c]<df_dsc["95%"])&(df[c]>df_dsc["5%"]),:]    
        plthlp.make_figures_for_df(df, name, OUT_DIR)
        if "onefig" in TESTS: break
        print('Outdoor figures saved to', OUT_DIR)

Outdoor figures saved to C:\Users\wsfm\OneDrive - Loughborough University\_Personal_Backup\ground_based_solar_irradiance\v_characterisation_output


In [50]:
df.groupby("month").agg(len)

,unnamed: 0,beam,diffuse,global,dt,hour,aoi,beam in plane,global minus diffuse,abs_err,cos_err
month,,,,,,,,,,,
3,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
